In [1]:
!pip install -U mxnet-cu101==1.7.0
!pip install d2l==0.17.2

     |███████████████████████████████▌| 834.1 MB 1.3 MB/s eta 0:00:09tcmalloc: large alloc 1147494400 bytes == 0x556fd0d24000 @  0x7f2860125615 0x556f973444cc 0x556f9742447a 0x556f973472ed 0x556f97438e1d 0x556f973bae99 0x556f973b59ee 0x556f97348bda 0x556f973bad00 0x556f973b59ee 0x556f97348bda 0x556f973b7737 0x556f97439c66 0x556f973b6daf 0x556f97439c66 0x556f973b6daf 0x556f97439c66 0x556f973b6daf 0x556f97349039 0x556f9738c409 0x556f97347c52 0x556f973bac25 0x556f973b59ee 0x556f97348bda 0x556f973b7737 0x556f973b59ee 0x556f97348bda 0x556f973b6915 0x556f97348afa 0x556f973b6c0d 0x556f973b59ee
     |████████████████████████████████| 846.0 MB 22 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 82 kB 580 kB/s 
     |████████████████████████████████| 20.1 MB 714 kB/s 
     |████████████████████████████████| 11.6 MB 37.8 MB/s 
     |██████

# Generating the Dataset

In [1]:
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

In [2]:
true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# Reading the Dataset

In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a Gluon data iterator."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[array([[-1.4548141 , -0.02985478],
        [-0.01892485, -1.5284138 ],
        [ 0.46009848, -0.20287366],
        [ 0.58428466,  0.35468408],
        [-0.82165414, -2.3175972 ],
        [-1.1976235 ,  1.6403017 ],
        [-1.6079371 , -0.4548878 ],
        [-0.39546987,  1.7430604 ],
        [-1.4096868 , -0.61659503],
        [ 0.29106006, -0.4088766 ]]), array([[ 1.4094979],
        [ 9.356497 ],
        [ 5.805376 ],
        [ 4.163606 ],
        [10.455184 ],
        [-3.7594023],
        [ 2.5222678],
        [-2.5096862],
        [ 3.449595 ],
        [ 6.160321 ]])]

# Defining the Model

In [6]:
# `nn` is an abbreviation for neural networks
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1))

# Initializing Model Parameters

In [7]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

# Defining the Loss Function

In [8]:
loss = gluon.loss.L2Loss()

# Defining the Optimization Algorithm

In [9]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# Training

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

epoch 1, loss 0.025146
epoch 2, loss 0.000090
epoch 3, loss 0.000051


In [11]:
w = net[0].weight.data()
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'error in estimating b: {true_b - b}')

error in estimating w: [ 8.229017e-04 -2.193451e-05]
error in estimating b: [0.00040388]
